In [2]:
import setup
!pwd

/home/ankit/Documents/development/ai/django-slack-bot/src/nbs


In [3]:
setup.init_django()

In [4]:
from django.conf import settings
settings.BASE_DIR

PosixPath('/home/ankit/Documents/development/ai/django-slack-bot/src')

In [5]:
%pip install llama-index upstash-vector llama-index-vector-stores-upstash

Note: you may need to restart the kernel to use updated packages.


In [6]:
django_pdf = "data/django.pdf"

In [7]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext
from llama_index.vector_stores.upstash import UpstashVectorStore

In [8]:
reader = SimpleDirectoryReader(input_files=[django_pdf])

In [9]:
import helpers
import openai
openai.api_key = helpers.config("OPENAI_API_KEY", default=None)

In [10]:
url=helpers.config("UPSTASH_VECTOR_URL", default=None)
token=helpers.config("UPSTASH_VECTOR_TOKEN", default=None)

In [11]:
data = reader.load_data()

In [12]:
data

[Document(id_='236247ce-0a8e-4293-88b2-a1acb9d6eea0', embedding=None, metadata={'page_label': '1', 'file_name': 'django.pdf', 'file_path': 'data/django.pdf', 'file_type': 'application/pdf', 'file_size': 10092684, 'creation_date': '2024-05-04', 'last_modified_date': '2024-05-04'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Django Documentation\nRelease 5.0.5.dev20240504093412\nDjango Software Foundation\nMay 04, 2024', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='2dac8f0a-3ed5-42de-ab95-52f80475a96d', embedding=None, metadata={'page_label': '2', 'file_name': 'django.pdf', 'file_path': 'data/django.pdf', 'file_type': 'application/pdf', 

In [16]:
import httpx
timeout = httpx.Timeout(600,connect = 600.0)
client = httpx.Client(timeout=timeout)


In [20]:
from upstash_vector import Index

upstash_index = Index(url = url,token = token,retries = 5, retry_interval=0.2)
upstash_index._client = client

In [21]:
vector_store = UpstashVectorStore(
    url=url, 
    token=token,
)
vector_store._index = upstash_index


In [22]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [23]:
index = VectorStoreIndex.from_documents(data, storage_context=storage_context, insert_batch_size=500)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
index_persist = VectorStoreIndex.from_vector_store(vector_store=vector_store)

In [ ]:
from llama_index.core.response_synthesizers.type import ResponseMode
query_engine = index_persist.as_query_engine()

In [ ]:
r = query_engine.query("What are the arguments for CharField?")
print(r)
